In [3]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import os
from datetime import datetime, timedelta
import time

In [32]:
def get_tweets(query, limit, columns):
  tweets = []
  count = 0
  for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    count += 1
    if count % 500 == 0:
      print(str(count) + " tweets downloaded...")

    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.content.replace('\n', '\\n').replace('\t', '\\t')])
  
  return pd.DataFrame(tweets, columns=columns)

def save_tweets_to_csv(tweet_df, path, filename):
  if not os.path.exists(path):
    os.makedirs(path)
  tweet_df.to_csv(path + '/' + filename)
  print('Saved tweets to ' + path + '/' + filename)
    
def collect_tweets(start_date, end_date, limit, columns, query, dir_path, filename):
    
    result_df = pd.DataFrame(columns=columns)
    starting_time = time.time()
    current_date = start_date
    while (current_date < end_date):
        next_day = current_date + timedelta(days=1)
        search_query = query + f" -filter:retweets -filter:links until:{str(next_day).split(' ')[0]}  since:{str(current_date).split(' ')[0]}"
        tweets = get_tweets(search_query, limit, columns)
        result_df = pd.concat([result_df, tweets])
        print('Appended ' + str(len(tweets)) + ' tweets from ' + str(current_date))
        print('Time elapsed: ' + str(time.time() - starting_time) + ' seconds.')
        current_date = next_day
    
    init_length = len(result_df)
    result_df.drop_duplicates(subset=['text'], inplace=True)
    print(f"{init_length - len(result_df)} duplicates dropped")

    save_tweets_to_csv(result_df, dir_path, filename)
    
    

In [33]:
start_date = datetime(2021, 12, 20)
end_date = datetime(2022, 5, 16)
columns = ["Date", "text"]

## Question 1

In [ ]:
collect_tweets(start_date, end_date, 500, columns, "russia ukraine lang:en", 'output/q1', 'all_tweets.csv')

## Question 2

### English

In [ ]:
collect_tweets(start_date, end_date, 250, columns, "putin -zelensky lang:en", 'output/q2', 'putin_english.csv')

In [ ]:
collect_tweets(start_date, end_date, 250, columns, "zelensky -putin lang:en", 'output/q2', 'zelensky_english.csv')

In [ ]:
collect_tweets(start_date, end_date, 250, columns, "nato -putin -zelensky lang:en", 'output/q2', 'nato_english.csv')

### Russian

In [ ]:
collect_tweets(start_date, end_date, 250, columns, "Путин -Зеленский lang:ru", 'output/q2', 'putin_russian.csv')

In [ ]:
collect_tweets(start_date, end_date, 250, columns, "Зеленский -Путин lang:ru", 'output/q2', 'zelensky_russian.csv')

In [ ]:
collect_tweets(start_date, end_date, 250, columns, "НАТО -Зеленский -Путин lang:ru", 'output/q2', 'nato_russian.csv')

## Question 3

In [ ]:
collect_tweets(start_date, end_date, 190, columns, "(russia OR ukraine) (to:FoxNews) lang:en", 'output/q3', 'fox_news.csv')

In [ ]:
collect_tweets(start_date, end_date, 190, columns, "(russia OR ukraine) (to:nytimes) lang:en", 'output/q3', 'new_york_times.csv')

In [ ]:
print(df)